In [1]:
import requests
import re
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import wordnet
import matplotlib.pyplot as plt

## Synonyms with Thesaurus API

In [ ]:
keys = ["production", "retail", "processing", "distribution"]
#no_keys = [ "consumer discretionary", "consumer staples", "communication services"]

In [ ]:
url = "https://www.dictionaryapi.com/api/v3/references/thesaurus/json/"
key= "?key=bc125025-b3b8-4b13-b8e8-c1ef37845bad"
syn_json = requests.get("https://www.dictionaryapi.com/api/v3/references/collegiate/json/retail?key=68bb26ab-7a34-4349-ae23-f4bc5b51aac2").json()

In [ ]:
syn_json

In [ ]:
def get_synonyms_api(key_word):
    url = "https://www.dictionaryapi.com/api/v3/references/thesaurus/json/"
    key= "?key=bc125025-b3b8-4b13-b8e8-c1ef37845bad"
    syn_json = requests.get(url+key_word+key).json()
    syn_list = syn_json[0]['meta']['syns']
    if len(syn_list) > 1:
        master = sum(syn_list, [])
    elif len(syn_list) == 1:
        master = syn_list[0]
    master.append(key_word)
    return master

In [ ]:
prod = get_synonyms_api(keys[0])
retail = get_synonyms_api(keys[1])
process = get_synonyms_api(keys[2])
distri =  get_synonyms_api(keys[3])


In [ ]:
retail

## Synonyms with NLP

In [ ]:
def get_synonyms(phr):
    phr_syn_lst = [phr]
    phr_syn_lst = [i.name().lower() for syn in wordnet.synsets(phr) for i in syn.lemmas() ]

    return (list(set(phr_syn_lst)))    

In [ ]:
get_synonyms("production")

In [ ]:
get_synonyms("retail")

## Algorithm

In [ ]:
df = pd.read_csv("scopus (4).csv")#, on_bad_lines='skip')
df.columns

In [ ]:
df.info()

In [ ]:
df = df.fillna("blank")

In [ ]:
df.head(2)

In [ ]:
non_alp = [';', ',', '(', ')', '[', ']', '.']
for i in non_alp:
    for col in ['Title','Abstract', 'Author Keywords', 'Index Keywords']:
        #df[col] = df[col].str.lower().replace(i,'')
        df[col] = df[col].map(lambda x: x.replace(i,''))

In [ ]:
df.head(2)

In [ ]:
df['content_sum'] = df['Abstract'] + df['Title'] + df['Author Keywords'] + df['Index Keywords']
df["content_sum"] = df["content_sum"].str.lower()
df['content_sum'].values[0].split()

In [ ]:
df_copy = df.copy()

In [ ]:
# Saving into a dictionary all of the syns
keys_dict = {}
for i in keys:
    keys_dict[i] = get_synonyms_api(i)
 
keys_dict

In [ ]:
# counting the keyworkds with their syns in all of the documents
for key in keys_dict.keys():
    df_copy[key] = 0
    for phr in keys_dict[key]:
        for i in range(len(df_copy)):
            if phr in (df_copy['content_sum'].values[i].split()):
                df_copy[key][i] += 1
    #df.loc[key, i] = count

In [ ]:
df_copy["Production"].sum()
sum_dic = {}
for i in keys:
    sum_dic[i] = df_copy[i].sum()
 
sum_dic

In [ ]:
# create a figure and set different background
fig = plt.figure(figsize=(5,5))
fig.patch.set_facecolor('white')
 
# Change color of text
plt.rcParams['text.color'] = 'black'
 
# Pieplot + circle on it
plt.pie(sum_dic.values(), labels=sum_dic.keys(), explode=(0, 0.1, 0, 0))
p=plt.gcf()
plt.show()

In [ ]:
df_copy.loc[df_copy["Production"] > 2]